In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup

def driversetup():
    options = webdriver.ChromeOptions()
    #run Selenium in headless mode
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    #overcome limited resource problems
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("lang=en")
    #open Browser in maximized mode
    options.add_argument("start-maximized")
    #disable infobars
    options.add_argument("disable-infobars")
    #disable extension
    options.add_argument("--disable-extensions")
    options.add_argument("--incognito")
    options.add_argument("--disable-blink-features=AutomationControlled")
    
    driver = webdriver.Chrome(options=options)

    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined});")

    return driver

In [2]:
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from bs4 import BeautifulSoup

def extract(url,year):
    #'https://www.wunderground.com/history/monthly/in/chennai/VOMM/date/2021-9'
    driver = webdriver.Chrome()
    driver.get(url)
    tables = WebDriverWait(driver,20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table")))
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    div = soup.find('div',{'class':'observation-table ng-star-inserted'})
    mon_list = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    date_list = []
    max_list = []
    avg_list = []
    min_list = []
    for tag in div:
        try:
            td = tag.find_all('td', {'class': 'ng-star-inserted'})
            count = 0
            for t in  td:
                row=t.find_all('tr',{'class': 'ng-star-inserted'})
                flag = 0
                for r in row:
                    txt = r.text.strip()
                    if txt in mon_list:
                        flag = 1
                    if flag==1:
                        date_list.append(txt)
                    if flag==0:
                        if txt.isalpha():
                            pass
                        else:
                            lt = txt.split()
                            max_list.append(lt[0])
                            avg_list.append(lt[1])
                            min_list.append(lt[2])

        except:
            pass


    val = date_list.pop(0)
    mon_idx = mon_list.index(val)+1
    for i in range(0,len(date_list)):
        if mon_idx<10:
            date_list[i] = date_list[i]+"-0"+str(mon_idx)+"-"+str(year)
        else:
            date_list[i] = date_list[i]+"-"+str(mon_idx)+"-"+str(year)
    
    temperature=[]
    humidity=[]
    count=0

    for i in avg_list:
        flag=0
        if i=='Avg':
            count=count+1
            flag=1
        if count==1 and flag==0:
            temperature.append(i)
        elif count==3 and flag==0:
            humidity.append(i)
    dict={'Date':date_list,'Temperature':temperature,'Humidity':humidity}
    df = pd.DataFrame(dict)
    driver.quit()
    return df

In [3]:
df1=extract('https://www.wunderground.com/history/monthly/in/chennai/VOMM/date/2021-9',2021)

In [4]:
df1.head()

,Date,Temperature,Humidity
0,1-09-2021,85.3,78.3
1,2-09-2021,82.9,79.1
2,3-09-2021,84.5,76.8
3,4-09-2021,83.0,77.3
4,5-09-2021,82.8,78.3


In [5]:
df1.shape

(30, 3)

In [6]:
frames = [df1, df1]
df1 = pd.concat(frames)
df1.shape

(60, 3)

In [7]:
#
df=pd.DataFrame(columns=["Date","Temperature","Humidity"])
link_lst = ['2021-9','2021-10','2021-11','2021-12','2022-1','2022-2','2022-3','2022-4','2022-5','2022-6','2022-7','2022-8']
for lnk in link_lst:
    url = 'https://www.wunderground.com/history/monthly/in/chennai/VOMM/date/'+lnk
    year = int(lnk[:4])
    df_new = extract(url,year)
    frames = [df, df_new]
    df = pd.concat(frames)

In [8]:
df.shape

(365, 3)

In [10]:
df.head()

,Date,Temperature,Humidity
0,1-09-2021,85.3,78.3
1,2-09-2021,82.9,79.1
2,3-09-2021,84.5,76.8
3,4-09-2021,83.0,77.3
4,5-09-2021,82.8,78.3


In [9]:
df.to_csv("Chennai_Weather.csv", sep=',', index=False)